In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(
    'data.csv'
)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232724 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

In [4]:
data.tail()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489
232724,Soul,Mint Condition,You Don't Have To Hurt No More,34XO9RwPMKjbvRry54QzWn,35,0.09730,0.758,323027,0.470,0.000049,G#,0.0836,-6.708,Minor,0.0287,113.897,4/4,0.479


# EDA:

In [5]:
data['genre'] = data['genre'].replace({
        "Children’s Music": "Children's Music",
        "Movie": "Soundtrack",
        "Opera": "Classical"
    })
data['genre'].unique()

array(['Soundtrack', 'R&B', 'A Capella', 'Alternative', 'Country',
       'Dance', 'Electronic', 'Anime', 'Folk', 'Blues', 'Classical',
       'Hip-Hop', "Children's Music", 'Rap', 'Indie', 'Pop', 'Reggae',
       'Reggaeton', 'Jazz', 'Rock', 'Ska', 'Comedy', 'Soul', 'World'],
      dtype=object)

In [6]:
from src.funcseda import music_eda
eda = music_eda(data)
eda.do_encoding()

,genre,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,item_id
135,1,Mary J. Blige,Be Without You - Kendu Mix,65,0.0830,0.724,246333,0.689,0.000000,2,0.3040,-5.922,0,0.1350,146,4,0.6930,135
136,1,Rihanna,Desperado,63,0.3230,0.685,186467,0.610,0.000000,0,0.1020,-5.221,0,0.0439,94,3,0.3230,136
137,1,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,62,0.0675,0.762,199520,0.520,0.000004,5,0.1140,-5.237,0,0.0959,75,4,0.0862,137
139,1,Olivia O'Brien,Love Myself,68,0.5960,0.653,213947,0.621,0.000000,11,0.0811,-5.721,0,0.0409,100,4,0.4660,139
141,1,Nao,Make It Out Alive (feat. SiR),64,0.6670,0.670,239147,0.649,0.000130,0,0.1090,-7.426,0,0.0875,148,4,0.6180,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227238,23,Herbie Hancock,Cantaloupe Island - Remastered 1999/Rudy Van G...,62,0.7490,0.515,333947,0.583,0.000294,0,0.1230,-8.182,0,0.0284,116,4,0.4150,227238
227743,23,Mura Masa,What If I Go? (feat. Bonzai),62,0.1940,0.560,195467,0.533,0.000014,7,0.0919,-7.804,1,0.3880,200,4,0.9080,227743
227839,23,Natalie Cole,Miss You Like Crazy,62,0.6330,0.596,234280,0.546,0.003810,2,0.0707,-6.251,1,0.0267,131,4,0.2550,227839
227951,23,Randy Crawford,One Day I'll Fly Away,64,0.7610,0.489,298027,0.358,0.018000,2,0.1060,-10.413,1,0.0302,78,4,0.4220,227951


In [7]:
data = eda.data

In [8]:
counts_dict = data['genre'].value_counts(dropna=False).to_dict()
sorted_by_key = dict(sorted(counts_dict.items(), reverse=False))
sorted_by_key

{1: 1445,
 3: 756,
 4: 615,
 6: 154,
 8: 901,
 9: 154,
 11: 2934,
 13: 3680,
 14: 1694,
 15: 48,
 16: 7182,
 17: 128,
 19: 150,
 20: 3330,
 21: 59,
 23: 627}

In [9]:
counts_dict = data['popularity'].value_counts(dropna=False).to_dict()
sorted_by_key = dict(sorted(counts_dict.items(), reverse=False))
sorted_by_key

{62: 2930,
 63: 2528,
 64: 2304,
 65: 2084,
 66: 2051,
 67: 1711,
 68: 1512,
 69: 1297,
 70: 1208,
 71: 997,
 72: 835,
 73: 826,
 74: 648,
 75: 521,
 76: 462,
 77: 397,
 78: 304,
 79: 253,
 80: 212,
 81: 149,
 82: 127,
 83: 86,
 84: 95,
 85: 72,
 86: 57,
 87: 43,
 88: 31,
 89: 19,
 90: 16,
 91: 23,
 92: 16,
 93: 10,
 94: 6,
 95: 7,
 96: 6,
 97: 8,
 98: 2,
 99: 3,
 100: 1}

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23857 entries, 135 to 228607
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             23857 non-null  int64  
 1   artist_name       23857 non-null  object 
 2   track_name        23857 non-null  object 
 3   popularity        23857 non-null  int64  
 4   acousticness      23857 non-null  float64
 5   danceability      23857 non-null  float64
 6   duration_ms       23857 non-null  int64  
 7   energy            23857 non-null  float64
 8   instrumentalness  23857 non-null  float64
 9   key               23857 non-null  int64  
 10  liveness          23857 non-null  float64
 11  loudness          23857 non-null  float64
 12  mode              23857 non-null  int64  
 13  speechiness       23857 non-null  float64
 14  tempo             23857 non-null  Int64  
 15  time_signature    23857 non-null  int64  
 16  valence           23857 non-null  float64


In [11]:
from src.this_mapping import supported_genres

# Создаём маппинг индекс → название жанра
idx_to_genre = {i: name for i, name in enumerate(supported_genres)}

# Декодируем числовой жанр в название
data['genre_name'] = data['genre'].map(idx_to_genre)

# Проверяем, нет ли NaN (некорректных жанров)
if data['genre_name'].isna().any():
    raise ValueError("Обнаружены некорректные значения жанра!")

# Группируем по жанру и считаем СУММАРНУЮ популярность (можно заменить на .mean() для средней)
genre_pop_sum = (
    data.groupby('genre_name')['popularity']
    .sum()  # или .mean(), если хочешь среднюю популярность трека в жанре
    .sort_values(ascending=False)
)

# Выводим как словарь (жанр → суммарная популярность)
genre_pop_dict = genre_pop_sum.to_dict()
for genre, total_pop in genre_pop_dict.items():
    print(f"{genre}")

Pop
Rap
Rock
Hip-Hop
Indie
R&B
Folk
Alternative
Soul
Country
Blues
Electronic
Jazz
Reggae
Ska
Classical


## Каждый значимый признак закодирован. ЕDA окончено

In [12]:
data.to_csv('edadata.csv', index=False)